# Amazon Bedrock으로 RAG 적용된 Chatbot 만들기

> *이 노트북은 SageMaker Studio 커널 **`Data Science 3.0`** 에서 잘 동작합니다.*

### Amazon Bedrock으로 챗봇을 구축하기 위한 랭체인 프레임워크
- 챗봇과 같은 대화형 인터페이스에서는 단기적 수준뿐만 아니라 장기적 수준에서도 이전 상호 작용을 기억하는 것이 매우 중요합니다.
- LangChain은 메모리 구성 요소를 두 가지 형태로 제공합니다.먼저 LangChain은 이전 채팅 메시지를 관리하고 조작하기 위한 도우미 유틸리티를 제공합니다.모듈식으로 설계되어 사용 방식에 관계없이 유용하게 사용할 수 있습니다.둘째, LangChain은 이러한 유틸리티를 체인에 통합하는 쉬운 방법을 제공합니다. 이를 통해 다양한 유형의 추상화를 쉽게 정의하고 상호 작용할 수 있으므로 강력한 챗봇을 쉽게 구축할 수 있습니다.
- RAG(Retrief-Augmented Generation)를 통해 외부 데이터로 모델의 기초 데이터를 보완합니다. LangChain의 Conversational RetrieveChain 클래스를 사용하여 챗봇과 RAG 기능을 한 번의 통화로 결합합니다.

### Prerequisites
- 몇 가지 추가 종속성이 필요합니다.
> FAISS : 벡터 임베딩 저장용 <br>
> iPyWidgets : 노트북의 대화형 UI 위젯용 <br>
> PyPDF : PDF 파일 처리용

In [ ]:
%pip install --quiet "faiss-cpu>=1.7,<2" "ipywidgets>=7,<8" langchain==0.0.304 "pypdf>=3.8,<4"

## 1. Amazon Bedrock Client 생성

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

## 2. 챗봇 (기본 - 컨텍스트 없음)

Anthropic의 Claude-v2 모델을 사용하여 한글이 지원되는 챗봇을 만들 것 입니다.

우리는 LangChain의 [CoversationChain](https://python.langchain.com/en/latest/modules/models/llms/integrations/bedrock.html?highlight=ConversationChain#using-in-a-conversation-chain)을 사용하여 시작합니다. 또한 메시지 저장을 위해 [ConversationBufferMemory](https://python.langchain.com/en/latest/modules/memory/types/buffer.html)를 사용합니다. 메시지 목록으로 기록을 얻을 수도 있습니다(채팅 모델에서 매우 유용합니다).

챗봇은 이전 상호작용을 기억해야 합니다. 대화 기억을 통해 우리는 그렇게 할 수 있습니다. 대화형 메모리를 구현하는 방법에는 여러 가지가 있습니다. LangChain의 맥락에서 이들은 모두 ConversationChain 위에 구축됩니다.



In [ ]:
from langchain.chains import ConversationChain
from langchain.llms.bedrock import Bedrock
from langchain.memory import ConversationBufferMemory

cl_llm = Bedrock(
    model_id="anthropic.claude-v2",
    client=boto3_bedrock,
    model_kwargs={"max_tokens_to_sample": 1000},
)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=cl_llm, verbose=True, memory=memory
)

print_ww(conversation.predict(input="안녕하세요?"))

### 결과 분석
[효과적인 Claude 프롬프트](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design)는  `\n\nHuman\n\nAassistant:` 로 끝나야 합니다. 이 문제를 해결해 보겠습니다.

Claude의 프롬프트 작성 방법에 대해 자세히 알아보려면 [Anthropic 문서](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design)를 확인하세요.

## 3. 프롬프트 템플릿(Langchain)을 이용한 챗봇

LangChain은 프롬프트를 쉽게 구성하고 작업할 수 있도록 여러 클래스와 기능을 제공합니다.

[PromptTemplate](https://python.langchain.com/en/latest/modules/prompts/getting_started.html) 클래스를 사용하여 f-string 템플릿에서 프롬프트를 구성하겠습니다.

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain import PromptTemplate

# turn verbose to true to see the full logs and documents
conversation= ConversationChain(
    llm=cl_llm, verbose=False, memory=ConversationBufferMemory() #memory_chain
)

# langchain prompts do not always work with all the models. This prompt is tuned for Claude

claude_prompt = PromptTemplate.from_template("""다음은 사람과 인공지능의 친근한 대화입니다. 
인공지능은 말이 많고 상황에 맞는 구체적인 세부 정보를 많이 제공합니다. 
인공지능은 모른다면 질문에 대한 대답은 솔직히 모른다고 말합니다.

Current conversation:
{history}


Human: {input}


Assistant:
""")


print("claude_prompt: \n", claude_prompt)

In [ ]:
conversation.prompt = claude_prompt

print_ww(conversation.predict(input="안녕하세요?"))

#### (1) 새로운 질문

모델이 초기 메시지로 응답했습니다. 몇 가지 질문을 해보겠습니다.

In [ ]:
print_ww(conversation.predict(input="새로운 정원을 시작하는 방법에 대한 몇 가지 팁을 알려주세요."))

#### (2) 질문을 토대로 작성

모델이 이전 대화를 이해할 수 있는지 확인하기 위해 정원이라는 단어를 언급하지 않고 질문해 보겠습니다.

In [ ]:
print_ww(conversation.predict(input="좋아요. 토마토에도 어울릴까요?"))

#### (3) 대화 마무리

In [ ]:
print_ww(conversation.predict(input="그게 다야, 고마워!"))

## 4. ipywidgets를 사용한 대화형 세션

다음 유틸리티 클래스를 사용하면 Claude와 보다 자연스러운 방식으로 상호 작용할 수 있습니다. 입력창에 질문을 적고 Claude의 답변을 받습니다. 그렇게 대화를 이어갈 수 있습니다.

In [ ]:
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        result = self.qa.run({'question': prompt })
                    else:
                        result = self.qa.run({'input': prompt }) #, 'history':chat_history})
                except:
                    result = "No answer"
                thinking.value=""
                print_ww(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

#### (1) 채팅을 시작해 보겠습니다. 다음 질문을 테스트할 수도 있습니다.

1. 농담 하나 해줘
2. 또 다른 농담을 들려주세요
3. 첫 번째 농담은 무엇이었나요?
4. 첫 번째 농담과 같은 주제로 또 다른 농담을 할 수 있나요?

위의 4가지를 순서대로 아래에 입력하시고, "Send" 버튼을 눌러 보세요.

In [ ]:
chat = ChatUX(conversation)
chat.start_chat()